# 오차역전파법
$$
f = e \times \left( (a \times b) + (c \times d) \right) 
$$
이 식에 대해서, 다음과 같은 순서로 풀 수 있다.
1. $a \times b = x$, $c \times d = y$ 계산
2. $x+y=z$ 계산
3. $e \times z= f$  

보통 우리가 푸는 방식으로 푸는 것을 **순전파(Forward propagation)**이라 한다.  
이를 반대 방향으로 하게 되면 어떨까?  
$$
\begin{align}
x &\triangleq a + b \\
y &\triangleq c + d \\
z &\triangleq x+y 
\end{align}
$$
위와 같이 정의한다고 하면, 반대 방식은 다음과 같다.  
1. $f = z \times e$ 계산
2. $z = x + y$ 계산
3. $x = a \times b$, $y = c \times d$ 계산  

이렇게, 순전파의 반대로 계산해 나가는 방식은 **역전파(Backward propagation)**이라 한다.  
굳이 이런 방식으로 계산하는 이유는, 미분 값을 효율적으로 구할 수 있기 때문이다.  

위의 예시를 이어가서, $a$의 변화에 따른 $f$의 변화량, 즉 $\frac{\partial f}{\partial a}$는 어떻게 계산할까?  
먼저, 다음 예시의 함성함수에서의 미분을 살펴보자.
$$
\begin{align}
z &= t^{2} \\
t &= x+y \\
\frac{\partial z}{\partial x} &= \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} \\
&= 2t \cdot 1
\end{align}
$$
이를 그대로 위의 예시에 적용하면 다음과 같다.  
$$
\begin{align}
\frac{\partial f}{\partial a} &= \frac{\partial f}{\partial z}\frac{\partial z}{\partial x}\frac{\partial x}{\partial a}
&= e \cdot 1 \cdot b = be
\end{align}
$$
$\frac{\partial f}{\partial b}$ 또한 살펴보자.
$$
\begin{align}
\frac{\partial f}{\partial b} *= \frac{\partial f}{\partial z}\frac{\partial z}{\partial x}\frac{\partial x}{\partial b}
&= e \cdot 1 \cdot a = ae
\end{align}
$$
이때, $\frac{\partial f}{\partial z}\frac{\partial z}{\partial x}$가 겹치는 것을 확인할 수 있는데, 역전파 방식의 경우, 이렇게 겹치는 계산을 활용하여, 중복된 계산 과정을 생략함으로써 효율성을 높인다.  
  
역전파의 계산은 단순히 2가지로, 곱셈과 덧셈이 있는데, 다음과 같다.
$$
\begin{align}
z &= x + y \\
\frac{\partial z}{\partial x} &= \frac{\partial z}{\partial y} = 1 
\end{align}
$$
$$
\begin{align}
z &= x \times y \\
\frac{\partial z}{\partial x} &= y,~~~\frac{\partial z}{\partial y} = x
\end{align}
$$
이렇게, $\frac{\partial z}{\partial x}$, $\frac{\partial x}{\partial a}$ 등의 각 미분 과정을 '계층'이라 한다.  



## 계층 구현
모든 계층(덧셈, 곱셈 계층)에 대해서 공통의 메서드 `foward()`, `backward()`를 갖도록 한다.  
이때 전자는 순전파, 후자는 역전파를 처리하는 메서드이다.  

### 곱셈 계층


In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

이에 대해서, 다음 계산을 위 코드를 사용하여 구해보자.  
$$
z = 1.1 \times (100 \times 2)
$$

In [ ]:
init_value = 100
mul_val_1 = 2
mul_val_2 = 1.1

# Layer instances
mul_layer_1 = MulLayer()
mul_layer_2 = MulLayer()

# Forward propagation   
out_1 = mul_layer_1.forward(init_value, mul_val_1)
result = mul_layer_2.forward(out_1, mul_val_2)

print("Forward Output:", result)

Forward Output: 220.00000000000003


In [4]:
dresult = 1
dout_2, dmul_val_2 = mul_layer_2.backward(dresult)
dout_1, dmul_val_1 = mul_layer_1.backward(dout_2)

print("Backward Output:", dout_1, dmul_val_1, dmul_val_2)

Backward Output: 2.2 110.00000000000001 200


### 덧셈 계층

In [5]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

덧셈 계층과 곱셈 계층을 이용하여, 다음 식을 계산해보자.
$$
\begin{align}
\text{result} &= c \times ((a \times a_{\text{mul}}) + (b \times b_{\text{mul}})) \\
a &= 100,~~a_{\text{mul}} = 2 \\
b &= 150,~~b_{\text{mul}} = 3 \\
c &= 1.1
\end{align}
$$

In [6]:
a = 100
a_mul = 2
b = 150
b_mul = 3
c = 1.1

# Layer instances
mul_layer_a = MulLayer()
mul_layer_b = MulLayer()
add_a_b_layer = AddLayer()
mul_layer_c = MulLayer()

# Forward propagation
a_out = mul_layer_a.forward(a, a_mul)
b_out = mul_layer_b.forward(b, b_mul)
a_b_out = add_a_b_layer.forward(a_out, b_out)
c_out = mul_layer_c.forward(a_b_out, c)

# backward propagation
dout = 1
da_b_out, dc = mul_layer_c.backward(dout)
da_out, db_out = add_a_b_layer.backward(da_b_out)
da, da_mul = mul_layer_a.backward(da_out)
db, db_mul = mul_layer_b.backward(db_out)

print("Final Output:", c_out)
print("Gradients:", da, da_mul, db, db_mul, dc)



Final Output: 715.0000000000001
Gradients: 2.2 110.00000000000001 3.3000000000000003 165.0 650


### 활성화 함수 계층

#### ReLU
ReLU는 다음과 같았다. 
$$
y = \max(x,0) = \begin{cases}
x & (x > 0) \\
0 & (x \leq 0)
\end{cases}
$$
이를 미분하면 다음과 같다.
$$
\frac{\partial y}{\partial x} = \begin{cases}
1 & (x > 0) \\
0 & (x \leq 0)
\end{cases}
$$


##### Code

In [7]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

#### Sigmoid 
Sigmoid 함수는 다음과 같다.
$$
y = \frac{1}{1+\exp(-x)}
$$

이는 다음과 같은 계산 과정을 가진다.
1. $a = x \times -1$
2. $b = \exp(a)$
3. $c = b + 1$
4. $d = \frac{1}{c}$

이를 역전파로 살펴보도록 한다.  
먼저 $y=\frac{1}{x}$ 를 미분하면 다음과 같다.
$$
\begin{align}
\frac{\partial y}{\partial x} &= - \frac{1}{x^{2}} \\
&= - y^{2}
\end{align}
$$
그 후, 덧셈, 즉, $c=b+1$에 대한 미분은 1이다.  
$b = \exp(a)$에 대한 미분은 다음과 같다.  
$$
\frac{\partial y}{\partial x} = \exp(x)
$$
그 후, 곱셈 $a = x \times -1$에 대한 미분은 $-1$이다.  
따라서, 이에 대한 역전파는 다음과 같다.
$$
\begin{align}
\frac{\partial y}{\partial x} &= \frac{\partial d}{\partial c} \frac{\partial c}{\partial b} \frac{\partial b}{\partial a} \frac{\partial a}{\partial x} \\
&= -y^{2} \times 1 \times \exp(a) \times -1 \\
&= y^{2}\exp(-x) \\
&= \frac{1}{(1+\exp(-x))^{2}}\exp(-x) \\
&= \frac{1}{1+\exp(-x)}\frac{\exp(-x)}{1+\exp(-x)} \\
&= y(1-y)
\end{align}
$$



#### Code

In [1]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx
        


#### Affine
신경망의 순전파에서 수행하는 행렬의 곱을 **어파인 변환 (Affine transformation)** 이라 한다.
먼저 다음 행렬식이 있다.
$$
\bold{O} = \bold{X} \cdot \bold{W}
$$
이는 다음과 같이 진행된다.  
1. $\bold{A}=\bold{X}\cdot\bold{W}$
2. $\bold{Y}=\bold{A}+\bold{B}$

이의 역전파를 생각해보도록 하자. 이에 대한 식은 다음과 같다.  
$$
\begin{align}
\frac{\partial L}{\partial \bold{X}} &= \frac{\partial L}{\partial \bold{Y}} \cdot \bold{W}^{T} \\
\frac{\partial L}{\partial \bold{W}} &= \bold{X}^{T} \cdot \frac{\partial L}{\partial \bold{Y}}
\end{align}
$$




##### Code

In [2]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

#### Softmax-with-Loss
softmax-with-Loss 계층은, softmax 계층과 Cross Entropy Error 계층으로 이루어져 있다.  
먼저, 입력 $\vec{a}$가 softmax 계층을 통과하여, $\vec{y}$를 출력하고, 그 후, Cross Entropy Error 계층으로 입력되어, 최종적으로 손실 $L$을 출력한다.  
입력 $\vec{y}$가 Cross Entropy Error 계층에 입력될 때 내부 가중치가 $\vec{t}$라면, 즉, 
$$
\text{CEE} = \sum_{n} t_{n} \log y_{n}
$$
이 경우, 이에 대한 역전파는 $\vec{a} - \vec{t}$로 간단하다. 

##### Code

In [9]:
import sys, os
sys.path.append(os.pardir)
from Functions.functions import cross_entropy_error
from Functions.layers import softmax


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None  
        self.t = None  
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

## 오차역전파법 구현
위에서의 계층을 조합하여, 전체적인 오차역전파법을 구현할 수 있다.  
다시 정리하여, 신경망 학습의 순서는 다음과 같다.  
1. 미니배치  
    훈련 데이터 중 일부를 무작위로 추출하여, 미니배치를 생성한다.
  
2. 기울기 산출  
    미니배치의 손실 함수 값을 줄이기 위해 각 가중치 파라미터의 기울기를 구한다.  
  
3. 파라미터 갱신  
    가중치 파라미터를 기울기 방향으로 학습률만큼 갱신한다.  
  
4. 앞선 1~3 단계를 반복한다.  
  


### Code

In [8]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from Functions.gradient import numerical_gradient
from Functions.layers import *
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

오차역전파법에 비해, 수치미분(순전파)는 속도가 느리지만, 구현에 있어서 난이도가 낮아 버그가 나타날 가능성이 낮다.   
이러한 이유로 수치미분은 주로 오차역전파법의 결과를 비교하여 에러가 없는지와 같은 목적으로 사용된다.  

### 비교 Code

In [14]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
y_batch = y_train[:3]

grad_numerical = network.numerical_gradient(x_batch, y_batch)
grad_backprop = network.gradient(x_batch, y_batch)

for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + " : " + str(diff))



W1 : 8.482589073248942e-05
b1 : 0.0006469382662078749
W2 : 0.0021908082462337264
b2 : 0.06666666678701484


### 학습 구현

#### Code

In [19]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iter_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iter_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    y_batch = y_train[batch_mask]
    
    grad = network.gradient(x_batch, y_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, y_train)
        test_acc = network.accuracy(x_test, y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("Train Accuracy:", train_acc, "Test Accuracy:", test_acc)



Train Accuracy: 0.14178333333333334 Test Accuracy: 0.1397
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09915 Test Accuracy: 0.1009
Train Accuracy: 0.09751666666666667 Test Accuracy: 0.0974
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
Train Accuracy: 0.09871666666666666 Test Accuracy: 0.098
